In [22]:
## Reads VTM format and outputs a vtm

import numpy as np
import os
import vtk as vtk
from sklearn.neighbors import NearestNeighbors

reader = vtk.vtkXMLMultiBlockDataReader()
reader.SetFileName('/home/soumya/Shared_Vbox/Data/MFIX/test_MFIX_50000.vtm')
reader.Update()

mpData = reader.GetOutput().GetBlock(0)
numPieces = mpData.GetNumberOfPieces()
neighborsize = 15

multiblock = vtk.vtkMultiBlockDataSet()

for i in range(0,numPieces):
    
    data = vtk.vtkPolyData()
    data = mpData.GetPiece(i)
    
    if data is not None:        
                
        numPts = data.GetNumberOfPoints()
        dataarr = []
        
        filtered = vtk.vtkPolyData()
        points  = vtk.vtkPoints()
        
        if numPts <= neighborsize:
            
            for i in range(0,numPts):
                pts =  data.GetPoint(i)
                points.InsertNextPoint(pts)
                
            filtered.SetPoints(points)  
            multiblock.SetBlock(i,filtered) 
                                
        else:
            
            for i in range(0,numPts):
                pts =  data.GetPoint(i)
                dataarr.insert(i,pts)

            nbrs = NearestNeighbors(n_neighbors=neighborsize, algorithm='ball_tree').fit(dataarr) 
            neighbors = nbrs.kneighbors(dataarr)[1]

            temparr = np.zeros(numPts)

            for i in range(0,numPts):

                if temparr[i] == 0:
                    kneighbors = nbrs.kneighbors(dataarr)[1][i]
                    temparr[i] = 2

                    for j in range(1,neighborsize):
                        temparr[kneighbors[j]] = 1 

            for i in range(0,numPts):
                if temparr[i] == 2:
                    pts =  data.GetPoint(i)
                    points.InsertNextPoint(pts)


            filtered.SetPoints(points)  
            multiblock.SetBlock(i,filtered)   
            
            
w = vtk.vtkXMLMultiBlockDataWriter()
w.SetInputData(multiblock)
filename = '/home/soumya/Shared_Vbox/MFIX_Out/test_out_287.vtm'
w.SetFileName(filename)
w.Write()

print 'Done'

Done


In [ ]:
#Reads vtp files individually and outputs a vtm

import vtk as vtk
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os

neighborsize = 15
multiblock = vtk.vtkMultiBlockDataSet()

ii=0
directory = '/home/soumya/Shared_Vbox/Data/MFIX/test_MFIX_50000/'
for filename in os.listdir(directory):
    if filename.endswith(".vtp"): 
        tstep = filename.split('_', 4 )[3]
        
        print 'procesing file: ' + str(tstep)
        
        datafile = directory + filename

        reader = vtk.vtkXMLPolyDataReader()
        reader.SetFileName(datafile)
        reader.Update()

        data = reader.GetOutput()
        numPts = data.GetNumberOfPoints()
        dataarr = []
        
        filtered = vtk.vtkPolyData()
        points  = vtk.vtkPoints()
        
        if numPts < neighborsize:
            
            for i in range(0,numPts):
                pts =  data.GetPoint(i)
                points.InsertNextPoint(pts)
                
            filtered.SetPoints(points)  
                                
        else:
            
            for i in range(0,numPts):
                pts =  data.GetPoint(i)
                dataarr.insert(i,pts)

            nbrs = NearestNeighbors(n_neighbors=neighborsize, algorithm='ball_tree').fit(dataarr) 
            neighbors = nbrs.kneighbors(dataarr)[1]

            temparr = np.zeros(numPts)

            for i in range(0,numPts):

                if temparr[i] == 0:
                    kneighbors = nbrs.kneighbors(dataarr)[1][i]
                    temparr[i] = 2

                    for j in range(1,neighborsize):
                        temparr[kneighbors[j]] = 1 

            for i in range(0,numPts):
                if temparr[i] == 2:
                    pts =  data.GetPoint(i)
                    points.InsertNextPoint(pts)


            filtered.SetPoints(points)  

        multiblock.SetBlock(ii,filtered)
        ii=ii+1

w = vtk.vtkXMLMultiBlockDataWriter()
w.SetInputData(multiblock)
filename = '/home/soumya/Shared_Vbox/MFIX_Out/test_out_287.vtm'
w.SetFileName(filename)
w.Write()

In [21]:
#Reads vtp files individually and outputs a single vtp file that has combined results

# While iterating over the points, the algorithm keeps the current point and removes its n-neighbors. 
# n is provided by users.
###########################################################################################################

import vtk as vtk
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os

neighborsize = 25
filtered = vtk.vtkPolyData()
points  = vtk.vtkPoints()

directory = '/home/soumya/Shared_Vbox/Data/MFIX_all/smooth-hg.nogas/plt_vtm_75/'
for filename in os.listdir(directory):
    if filename.endswith(".vtp"): 
        tstep = filename.split('_', 5 )[2]
        
        print 'procesing tstep: ' + str(tstep)
        
        datafile = directory + filename

        reader = vtk.vtkXMLPolyDataReader()
        reader.SetFileName(datafile)
        reader.Update()

        data = reader.GetOutput()
        numPts = data.GetNumberOfPoints()
        dataarr = []
       
        if numPts < neighborsize:
            
            for i in range(0,numPts):
                pts =  data.GetPoint(i)
                points.InsertNextPoint(pts)
                
        else:
            
            for i in range(0,numPts):
                pts =  data.GetPoint(i)
                dataarr.insert(i,pts)

            nbrs = NearestNeighbors(n_neighbors=neighborsize, algorithm='ball_tree').fit(dataarr) 
            neighbors = nbrs.kneighbors(dataarr)[1]

            temparr = np.zeros(numPts)

            for i in range(0,numPts):

                if temparr[i] == 0:
                    kneighbors = nbrs.kneighbors(dataarr)[1][i]
                    temparr[i] = 2
                    pts =  data.GetPoint(i)
                    points.InsertNextPoint(pts)

                    for j in range(1,neighborsize):
                        temparr[kneighbors[j]] = 1 
        
filtered.SetPoints(points)          

w = vtk.vtkXMLPolyDataWriter()
w.SetInputData(filtered)
filename = '/home/soumya/Shared_Vbox/MFIX_Out/sampled_out_75.vtp'
w.SetFileName(filename)
w.Write()

In [24]:
# Distance based Poisson's dart throwing sampling

# While iterating over the points, the algorithm keeps the current point and removes its n-neighbors if they are not
# within a predefined distance threshold. n is provided by users.
###########################################################################################################

#Reads vtp files individually and outputs a single vtp file that has combined results
## Uses a distance to discard points

import vtk as vtk
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os
import math

neighborsize = 25
filtered = vtk.vtkPolyData()
points  = vtk.vtkPoints()

TH = 0.01
tstep=150

directory = '/home/soumya/Shared_Vbox/Data/MFIX_all/smooth-hg.nogas/plt_vtm_' + str(tstep) + '/'
for filename in os.listdir(directory):
    if filename.endswith(".vtp"): 
        #tstep = filename.split('_', 5 )[2]
        
        datafile = directory + filename
        reader = vtk.vtkXMLPolyDataReader()
        reader.SetFileName(datafile)
        reader.Update()

        data = reader.GetOutput()
        numPts = data.GetNumberOfPoints()
        dataarr = []
       
        if numPts < neighborsize:
            
            for i in range(0,numPts):
                pts =  data.GetPoint(i)
                points.InsertNextPoint(pts)
                
        else:
            
            for i in range(0,numPts):
                pts =  data.GetPoint(i)
                dataarr.insert(i,pts)

            nbrs = NearestNeighbors(n_neighbors=neighborsize, algorithm='ball_tree').fit(dataarr) 
            neighbors = nbrs.kneighbors(dataarr)[1]

            temparr = np.zeros(numPts)

            for i in range(0,numPts):

                if temparr[i] == 0:
                    kneighbors = nbrs.kneighbors(dataarr)[1][i]
                    temparr[i] = 2
                    pts =  data.GetPoint(i)
                    points.InsertNextPoint(pts)

                    for j in range(1,neighborsize):
                        currentpts = data.GetPoint(kneighbors[j])
                        dist = math.sqrt(pow((pts[0]-currentpts[0]),2) + pow((pts[1]-currentpts[1]),2) + pow((pts[2]-currentpts[2]),2))
                        
                        if dist <= TH:
                            temparr[kneighbors[j]] = 1 
        
filtered.SetPoints(points)  

print filtered.GetNumberOfPoints()

w = vtk.vtkXMLPolyDataWriter()
w.SetInputData(filtered)
filename = '/home/soumya/Shared_Vbox/MFIX_Out/Poisson_sampling/Sampled_out_' + str(tstep) + '.vtp'
w.SetFileName(filename)
w.Write()

In [20]:
## Regular particle sampling
## Serial version. All time steps are processed sequentially

#Reads vtp files individually and outputs a single vtp file that has combined results
## Uses a distance to discard points

import vtk as vtk
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
import os
import math

percentage_TH = 10

# # generate the random id for regular sampling
random_id = random.randint(0,percentage_TH-1)
#print random_id

for tstep in range(0,824):
    
    filtered = vtk.vtkPolyData()
    points  = vtk.vtkPoints()

    directory = '/home/soumya/Shared_Vbox/Data/MFIX_all/smooth-hg.nogas/plt_vtm_' + str(tstep) + '/'
    #directory = '/home/soumya/Shared_Vbox/Data/MFIX_all/smooth-hg.nogas/plt_vtm_150/'
    for filename in os.listdir(directory):
        if filename.endswith(".vtp"): 
            #tstep = filename.split('_', 5 )[2]

            datafile = directory + filename

            reader = vtk.vtkXMLPolyDataReader()
            reader.SetFileName(datafile)
            reader.Update()

            data = reader.GetOutput()
            numPts = data.GetNumberOfPoints()

            if numPts < percentage_TH:

                pts =  data.GetPoint(0)
                points.InsertNextPoint(pts)

            else:

                for i in range(0,numPts,percentage_TH):
                    point_id = i+random_id

                    if (point_id < numPts):
                        pts =  data.GetPoint(point_id)
                        points.InsertNextPoint(pts)

    filtered.SetPoints(points)  

    w = vtk.vtkXMLPolyDataWriter()
    w.SetInputData(filtered)
    filename = '/home/soumya/Shared_Vbox/MFIX_Out/regular_sampled_out/Sampled_out_' + str(tstep) + '.vtp'
    w.SetFileName(filename)
    w.Write()

    print 'Done timestep: ' + str(tstep) 

Done timestep: 0
Done timestep: 1
Done timestep: 2
Done timestep: 3
Done timestep: 4
Done timestep: 5
Done timestep: 6
Done timestep: 7
Done timestep: 8
Done timestep: 9
Done timestep: 10
Done timestep: 11
Done timestep: 12
Done timestep: 13
Done timestep: 14
Done timestep: 15
Done timestep: 16
Done timestep: 17
Done timestep: 18
Done timestep: 19
Done timestep: 20
Done timestep: 21
Done timestep: 22
Done timestep: 23
Done timestep: 24
Done timestep: 25
Done timestep: 26
Done timestep: 27
Done timestep: 28
Done timestep: 29
Done timestep: 30
Done timestep: 31
Done timestep: 32
Done timestep: 33
Done timestep: 34
Done timestep: 35
Done timestep: 36
Done timestep: 37
Done timestep: 38
Done timestep: 39
Done timestep: 40
Done timestep: 41
Done timestep: 42
Done timestep: 43
Done timestep: 44
Done timestep: 45
Done timestep: 46
Done timestep: 47
Done timestep: 48
Done timestep: 49
Done timestep: 50
Done timestep: 51
Done timestep: 52
Done timestep: 53
Done timestep: 54
Done timestep: 55
Do

KeyboardInterrupt: 

In [27]:
## Regular particle sampling
## Parallel multithreaded version using joblib and multiprocessing. All time steps are processed in parallel.

# Reads vtp files individually and outputs a single vtp file that has combined results

import vtk as vtk
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
import os
import math
from joblib import Parallel, delayed
import multiprocessing

percentage_TH = 10

# # generate the random id for regular sampling
random_id = random.randint(0,percentage_TH-1)

def process_one_tstep(tstep):
    filtered = vtk.vtkPolyData()
    points  = vtk.vtkPoints()

    directory = '/home/soumya/Shared_Vbox/Data/MFIX_all/smooth-hg.nogas/plt_vtm_' + str(tstep) + '/'
    for filename in os.listdir(directory):
        if filename.endswith(".vtp"): 

            datafile = directory + filename
            reader = vtk.vtkXMLPolyDataReader()
            reader.SetFileName(datafile)
            reader.Update()

            data = reader.GetOutput()
            numPts = data.GetNumberOfPoints()

            if numPts < percentage_TH:

                pts =  data.GetPoint(0)
                points.InsertNextPoint(pts)

            else:

                for i in range(0,numPts,percentage_TH):
                    point_id = i+random_id

                    if (point_id < numPts):
                        pts =  data.GetPoint(point_id)
                        points.InsertNextPoint(pts)

    filtered.SetPoints(points)  

    w = vtk.vtkXMLPolyDataWriter()
    w.SetInputData(filtered)
    filename = '/home/soumya/Shared_Vbox/MFIX_Out/regular_sampled_out/Sampled_out_' + str(tstep) + '.vtp'
    w.SetFileName(filename)
    w.Write()

    print 'Done timestep: ' + str(tstep)
    
inputs = range(291,824)    
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(process_one_tstep)(j) for j in inputs)    

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [95]:
## Regular particle sampling
## Parallel multithreaded version using joblib and multiprocessing. All time steps are processed in parallel.

# Reads vtm files and outputs vtm files.

import vtk as vtk
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
import os
import math
from joblib import Parallel, delayed
import multiprocessing
import time

percentage_TH = 10

# generate the random id for regular sampling
random_id = random.randint(0,percentage_TH-1)
print random_id
#random_id = 0

def process_one_tstep(tstep):
    
    reader = vtk.vtkXMLMultiBlockDataReader()
    filename = '/home/soumya/Shared_Vbox/Data/MFIX_all/smooth-hg.nogas/plt_vtm_' + str(tstep) + '.vtm'
    reader.SetFileName(filename)
    reader.Update()
    
    ## Get input multiblock data from the reader
    mpData = reader.GetOutput().GetBlock(0)    
    
    ## create output multiblovk dataset
    multiblock = vtk.vtkMultiBlockDataSet()  
    numPieces = mpData.GetNumberOfPieces()
    multiblock.SetNumberOfBlocks(numPieces)  
    count=0
    
    ## loop over each block and sample regularly
    for i in range(numPieces):
        
        filtered = vtk.vtkPolyData()
        points  = vtk.vtkPoints()
    
        data = mpData.GetPiece(i)
        
        if data is not None:
            numPts = data.GetNumberOfPoints()

            if numPts < percentage_TH:
                pts =  data.GetPoint(0)
                points.InsertNextPoint(pts)
                
                filtered.SetPoints(points)  
                multiblock.SetBlock(count,filtered)

            else:
                for j in range(0,numPts-percentage_TH,percentage_TH):
                    
                    #random_id = random.randint(0,percentage_TH-1)
                    point_id = j+random_id
                    
                    pts =  data.GetPoint(point_id)
                    points.InsertNextPoint(pts)

                filtered.SetPoints(points)  
                multiblock.SetBlock(count,filtered)
                
            count=count+1    
                
    w = vtk.vtkXMLMultiBlockDataWriter()
    filename = '/home/soumya/Shared_Vbox/MFIX_Out/regular_sampled_out_vtm/Sampled_out_' + str(tstep) + '.vtm'
    w.SetFileName(filename)
    w.SetInputData(multiblock)
    w.Write()
    
## for running just one timestep uncomment it
start = time.time()
process_one_tstep(777) # provide tstep here
end = time.time()
print end - start

# ## for running multithreaded
# inputs = range(626,824)    
# num_cores = multiprocessing.cpu_count()
# Parallel(n_jobs=num_cores)(delayed(process_one_tstep)(i) for i in inputs)     

2
5.52473211288


In [122]:
#particle_entropy_time_series.py

import vtk
import numpy as np
import sys
import os
import math
import time
import scipy.cluster.hierarchy as hc
from matplotlib import pyplot as plt
from mpi4py import MPI
from sklearn.cluster import KMeans


def compute_entropy_spatial(in_file, outfile, range_min, range_max):
    reader = vtk.vtkXMLMultiBlockDataReader()
    # reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(in_file)
    # reader.UpdatePiece(comm.Get_rank(), comm.Get_size(), 0)
    reader.Update()

    # print("total points:",data.GetNumberOfPoints(),data.GetNumberOfElements(0))
    
    ### For Ayan
#     mpData = reader.GetOutput().GetBlock(0)
#     numPieces = mpData.GetNumberOfPieces()
#     print("pieces:", numPieces)
    
     ### For Soumya
    mpData = reader.GetOutput()
    numPieces = mpData.GetNumberOfBlocks()
    #print("pieces:", numPieces)

    multiblock = vtk.vtkMultiBlockDataSet()
    multiblock.SetNumberOfBlocks(numPieces)

    ## create a dictionary for storing the sparse histogram
    sp_hist = dict()
    n_bins = [32,4,4]


    pc_cnt =0

    for i in range(numPieces):
        #data = mpData.GetPiece(i)
        data = mpData.GetBlock(i)

        if data is not None:
            filtered = vtk.vtkPolyData()
            Points = vtk.vtkPoints()

            pts = data.GetPoints()

            tot_pts = data.GetNumberOfPoints()
            feat_arr = np.zeros((tot_pts, 3))

            for i in range(tot_pts):
                loc = pts.GetPoint(i)
                feat_arr[i, :] = np.asarray(loc)

                key_0 = min(int(n_bins[0]*(range_max[0]-np.asarray(loc)[0])/(range_max[0]-range_min[0])),n_bins[0]-1)
                key_1 = min(int(n_bins [1]* (range_max[1] - np.asarray(loc)[1]) / (range_max[1] - range_min[1])),
                            n_bins[1] - 1)
                key_2 = min(int(n_bins[2] * (range_max[2] - np.asarray(loc)[2]) / (range_max[2] - range_min[2])),
                            n_bins[2] - 1)
                #print(key_0,key_1,key_2)
                ## remove hard code
                if key_0<10:
                    key_str_0 = '0'+str(key_0)
                else:
                    key_str_0 = str(key_0)
                if key_1 < 10:
                    key_str_1 = '0' + str(key_1)
                else:
                    key_str_1 = str(key_1)
                if key_2 < 10:
                    key_str_2 = '0' + str(key_2)
                else:
                    key_str_2 = str(key_2)
                key_str = key_str_0+key_str_1+key_str_2
                if key_str not in sp_hist:
                    sp_hist[key_str]=1
                else :
                    sp_hist[key_str]=sp_hist[key_str]+1



            pc_cnt = pc_cnt + 1

    #print("total non-zero pieces:",pc_cnt)
    #print(sp_hist)
    tot_N = 0
    for k, v in sp_hist.items():
        #print(k, v)
        tot_N = tot_N + v
    #print("total points are:",tot_N)
    ent = 0
    for k, v in sp_hist.items():
        cur_val = 1.0*v/tot_N
        ent = ent -  cur_val* math.log(cur_val, 2)
    print("total entropy is :", ent)

    return ent

# all_ent = np.loadtxt('entropy_list.txt')
# plt.figure()
# plt.plot(all_ent)
# plt.show()


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

#print("rank",rank)
#print("size",size)

## first try to get the whole range

# read this pvtp file
# path = '/media/biswas/29d42253-dbfd-4e49-bc64-61662de409e0/MFIX/smooth-hg.nogas/'
# eb_file = 'eb.vtp'

# read this pvtp file for Soumya's machine
path = '/home/soumya/Shared_Vbox/Data/MFIX_all/smooth-hg.nogas/grid/'
eb_file = 'eb.vtp'

poly_reader = vtk.vtkXMLPolyDataReader()
poly_reader.SetFileName(path+eb_file)
poly_reader.Update()

data = poly_reader.GetOutput()

#print("total points:",data.GetNumberOfPoints(),data.GetNumberOfElements(0))

pts = data.GetPoints()

#pt_data = data.GetPointData().GetArray("mass").GetTuple1(0)

#print(pts.GetPoint(0))

tot_pts = data.GetNumberOfPoints()
feat_arr = np.zeros((tot_pts,3))

for i in range(tot_pts):
    loc = pts.GetPoint(i)
    feat_arr[i,:] = np.asarray(loc)

range_min = np.min(feat_arr,axis=0)
range_max = np.max(feat_arr,axis=0)

#print("range:",range_min,range_max)

# ## now read the data and compute entropy on that
# file_name1 = "/media/biswas/29d42253-dbfd-4e49-bc64-61662de409e0/MFIX/smooth-hg.nogas/vtm_format/"
# file_name2 = ".vtm"
# name_string = 'dat_timestep_824' #'test_MFIX_50000'

# in_file = file_name1+ name_string +file_name2
# entropy = compute_entropy_spatial(in_file, i, range_min, range_max)

# entropy_list = []
# for i in range(824):
#     name_string = 'dat_timestep_'+str(i)

#     in_file = file_name1+ name_string +file_name2
#     print("opening file:",in_file)
#     entropy = compute_entropy_spatial(in_file, i, range_min, range_max)
#     entropy_list.append(entropy)

# np.savetxt('entropy_list.txt',entropy_list)


## now read the data and compute entropy on that for Soumya's machine
file_name1 = "/home/soumya/Shared_Vbox/MFIX_Out/regular_sampled_out_vtm/"
file_name2 = ".vtm"
name_string = 'Sampled_out_823' #'test_MFIX_50000'

in_file = file_name1+ name_string +file_name2
#print("opening file:",in_file)
entropy = compute_entropy_spatial(in_file, i, range_min, range_max)

entropy_list = []
for i in range(824):
    name_string = 'Sampled_out_'+str(i)

    in_file = file_name1+ name_string +file_name2
    #print("opening file:",in_file)
    entropy = compute_entropy_spatial(in_file, i, range_min, range_max)
    entropy_list.append(entropy)

np.savetxt('entropy_list_regular.txt',entropy_list)


# Points = vtk.vtkPoints()
#
# for i in os.listdir(input_path):
#     # read files
#     filename, file_extension = os.path.splitext(i)
#     print("the file name is :",filename)
#     if file_extension != ".vtm":
#         continue
#     #print("opening", i)
#     # write the actual code here
#
#     sample_fraction = 0.1
#     in_file = input_path + i
#
#     #sample_data(in_file, i, sample_fraction)
#     #save_screenshot(in_file, 'out')
#     compute_entropy_spatial(in_file, i, range_min, range_max)
#     break

('total entropy is :', 5.065230956470404)
('total entropy is :', 5.3453747239849925)
('total entropy is :', 5.345947011254841)
('total entropy is :', 5.421546388840487)
('total entropy is :', 5.407911180903638)
('total entropy is :', 5.380443051413538)
('total entropy is :', 5.402198811389117)
('total entropy is :', 5.420811536643053)
('total entropy is :', 5.402575004251124)
('total entropy is :', 5.407636971533471)
('total entropy is :', 5.404394275283045)
('total entropy is :', 5.419701912186252)
('total entropy is :', 5.432627963777213)
('total entropy is :', 5.411398986766943)
('total entropy is :', 5.363574015713501)
('total entropy is :', 5.376976116729755)
('total entropy is :', 5.362316811713713)
('total entropy is :', 5.3185725503767225)
('total entropy is :', 5.290468632093835)
('total entropy is :', 5.270099429644028)
('total entropy is :', 5.163432712756354)
('total entropy is :', 5.161994586200345)
('total entropy is :', 5.111876369088)
('total entropy is :', 5.2006420244

('total entropy is :', 5.580668101152238)
('total entropy is :', 5.587122729132235)
('total entropy is :', 5.577057140456278)
('total entropy is :', 5.58653717515711)
('total entropy is :', 5.580997975534322)
('total entropy is :', 5.583704859183483)
('total entropy is :', 5.582552139276619)
('total entropy is :', 5.582147765411883)
('total entropy is :', 5.584413892961156)
('total entropy is :', 5.575375002462968)
('total entropy is :', 5.58355006609033)
('total entropy is :', 5.585665369064036)
('total entropy is :', 5.591207404650611)
('total entropy is :', 5.583632338922098)
('total entropy is :', 5.59423141330337)
('total entropy is :', 5.584946199916429)
('total entropy is :', 5.576783857719283)
('total entropy is :', 5.586547646550354)
('total entropy is :', 5.5820994324564035)
('total entropy is :', 5.577980169900088)
('total entropy is :', 5.588148421133021)
('total entropy is :', 5.583326520389917)
('total entropy is :', 5.587020998804396)
('total entropy is :', 5.58776839923

('total entropy is :', 5.587015688303876)
('total entropy is :', 5.598072822826782)
('total entropy is :', 5.587773185792559)
('total entropy is :', 5.5861627199676995)
('total entropy is :', 5.582606456188975)
('total entropy is :', 5.58359969394713)
('total entropy is :', 5.577817865590048)
('total entropy is :', 5.578305286078395)
('total entropy is :', 5.582919673727506)
('total entropy is :', 5.584287461615454)
('total entropy is :', 5.58183981362229)
('total entropy is :', 5.590814613391692)
('total entropy is :', 5.5878261427917435)
('total entropy is :', 5.5799694324147815)
('total entropy is :', 5.588250625875854)
('total entropy is :', 5.583586549603656)
('total entropy is :', 5.588068451051218)
('total entropy is :', 5.585977059956667)
('total entropy is :', 5.585908609708793)
('total entropy is :', 5.570292422724584)
('total entropy is :', 5.58397964994872)
('total entropy is :', 5.587674884608149)
('total entropy is :', 5.579209021968763)
('total entropy is :', 5.571702831

('total entropy is :', 5.514454439135382)
('total entropy is :', 5.51024649917698)
('total entropy is :', 5.503511490583166)
('total entropy is :', 5.510070012864383)
('total entropy is :', 5.504320783297585)
('total entropy is :', 5.505178125089027)
('total entropy is :', 5.503073763833707)
('total entropy is :', 5.501231478583305)
('total entropy is :', 5.497828291245322)
('total entropy is :', 5.499792929915813)
('total entropy is :', 5.4944665754584205)
('total entropy is :', 5.495744947475829)
('total entropy is :', 5.500395918186557)
('total entropy is :', 5.495053817043961)
('total entropy is :', 5.494505537927624)
('total entropy is :', 5.49927947642813)
('total entropy is :', 5.490548925457372)
('total entropy is :', 5.490276229348747)
('total entropy is :', 5.500451652551719)
('total entropy is :', 5.49517643055751)
('total entropy is :', 5.489297355275731)
('total entropy is :', 5.493284776874427)
('total entropy is :', 5.484033764184538)
('total entropy is :', 5.48687182984

('total entropy is :', 5.250050434507432)
('total entropy is :', 5.239570917875789)
('total entropy is :', 5.244746940416454)
('total entropy is :', 5.242130146054751)
('total entropy is :', 5.234597453045837)
('total entropy is :', 5.232430731960458)
('total entropy is :', 5.214365848371378)
('total entropy is :', 5.2174367356173805)
('total entropy is :', 5.2103932530777906)
('total entropy is :', 5.210399548236654)
('total entropy is :', 5.190901683948901)
('total entropy is :', 5.189752945633417)
('total entropy is :', 5.189337030772993)
('total entropy is :', 5.186465443711859)
('total entropy is :', 5.168318554671569)
('total entropy is :', 5.171264670352584)
('total entropy is :', 5.15850286286107)
('total entropy is :', 5.1529824249255896)
('total entropy is :', 5.150853274235178)
('total entropy is :', 5.145704707350205)
('total entropy is :', 5.139321650969943)
('total entropy is :', 5.128910384104125)
('total entropy is :', 5.1265562533467826)
('total entropy is :', 5.114636